<h1><center>LSTM Language Detection</center></h1>
<h3><center>CSE 6240 - Websearch and Text Mining</center></h3>

<h7><center>Adam Lieberman, Garrett Mallory, Ravish Chawla</center></h7>
<h7><center>April 25, 2017</center></h7>

1) make 80/20 split on end and frn languages. (lower case all letters)
2) create two lstm models (size 128)
3) train 5 epochs, 0.2 validation split on the training samples 
4) 
4) train on 100 examples of 5 character strings from text with the right label.

pass new word through lstm to get log probability. then do generative log ratio test. y = sign(pe/pj -1) to get the label. Keep scores for ROC.

to get log probability, need p(t), p(r|t), p(u|tr) ... etc. Do this by passing in t, then tr then tru .... This gives you the most likely next character distribution so index into that to get the prob of "U" from "tr".

pass both sets of 100 test (all 200) through both networks for the probability part


In [27]:
from __future__ import print_function;
import sys;
import random;
import numpy as np;

from keras.models import Sequential;
from keras.layers import Dense, Activation;
from keras.layers import LSTM;
from keras.optimizers import RMSprop;
from keras.utils.data_utils import get_file;
from sklearn.cross_validation import train_test_split;

In [2]:
english_text = open('eng.txt').read().lower()
french_text = open('frn.txt').read().lower()

print('English corpus length:', len(english_text))
print('French corpus length:', len(french_text))

English corpus length: 10746
French corpus length: 12009


In [3]:
english_chars = sorted(list(set(english_text)))
french_chars = sorted(list(set(french_text)))

english_char_map = dict((c, i) for i, c in enumerate(english_chars))
french_char_map = dict((c, i) for i, c in enumerate(french_chars))

english_char_map_inverse = dict((i, c) for i, c in enumerate(english_chars))
french_char_map_inverse = dict((i, c) for i, c in enumerate(french_chars))

In [5]:
print('English character count:', len(english_chars))
print('French character count:', len(french_chars))
for c in french_chars:
    if c not in english_chars:
        print("not present:", c)

English character count: 43
French character count: 41
not present: '


In [7]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3

english_sentences = []
english_next_chars = []
for i in range(0, len(english_text) - maxlen, step):
    english_sentences.append(english_text[i: i + maxlen])
    english_next_chars.append(english_text[i + maxlen])

french_sentences = []
french_next_chars = []
for i in range(0, len(french_text) - maxlen, step):
    french_sentences.append(french_text[i: i + maxlen])
    french_next_chars.append(french_text[i + maxlen])
    
print('nb English sequences:', len(english_sentences))
print('nb French sequences:', len(french_sentences))

nb English sequences: 3569
nb French sequences: 3990


In [34]:
print('Vectorization...')

larger_len = max(len(english_chars), len(french_chars));

english_x = np.zeros((len(english_sentences), maxlen, larger_len), dtype=np.bool)
english_y = np.zeros((len(english_sentences), larger_len), dtype=np.bool)
for i, sentence in enumerate(english_sentences):
    for t, char in enumerate(sentence):
        english_x[i, t, english_char_map[char]] = 1
    english_y[i, english_char_map[english_next_chars[i]]] = 1
    
    
french_x = np.zeros((len(french_sentences), maxlen, larger_len), dtype=np.bool)
french_y = np.zeros((len(french_sentences), larger_len), dtype=np.bool)
for i, sentence in enumerate(french_sentences):
    for t, char in enumerate(sentence):
        french_x[i, t, french_char_map[char]] = 1
    french_y[i, french_char_map[french_next_chars[i]]] = 1

print("Finished!")

Vectorization...
Finished!


In [85]:
english_train_x, english_test_x, english_train_y, english_test_y = train_test_split(english_x, english_y, test_size=0.2, random_state=1024);
french_train_x, french_test_x, french_train_y, french_test_y = train_test_split(french_x, french_y, test_size=0.2, random_state=1024);

In [86]:
print('English Shapes');
print(english_train_x.shape);
print(english_train_y.shape);
print(english_test_x.shape);
print(english_test_y.shape);
print()
print('French Shapes');
print(french_train_x.shape);
print(french_train_y.shape);
print(french_test_x.shape);
print(french_test_y.shape);

English Shapes
(2855, 40, 43)
(2855, 43)
(714, 40, 43)
(714, 43)

French Shapes
(3192, 40, 43)
(3192, 43)
(798, 40, 43)
(798, 43)


In [94]:
# build the model: a single LSTM
def build_model(chars):
    print('Build model...')
    model = Sequential()
    model.add(LSTM(128, input_shape=(maxlen, larger_len)))
    model.add(Dense(larger_len))
    model.add(Activation('softmax'))
    
    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

english_model = build_model(english_chars)
french_model = build_model(french_chars);

Build model...


In [95]:
history_english = english_model.fit(english_train_x, english_train_y,
                    #batch_size=128, epochs=5);
history_french = french_model.fit(french_train_x, french_train_y,
                    batch_size=128, epochs=5);

Epoch 1/5
3192/3192 [==============================] - 2s - loss: 2.9715     
Epoch 2/5
3192/3192 [==============================] - 2s - loss: 2.5878     
Epoch 3/5
3192/3192 [==============================] - 2s - loss: 2.2998     
Epoch 4/5
3192/3192 [==============================] - 2s - loss: 2.0953     
Epoch 5/5
3192/3192 [==============================] - 2s - loss: 1.9543     


In [96]:
sample = english_test_x[0];

In [102]:
pred_0 = english_model.predict(np.reshape(sample, (1, 40, 43)));

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False], dtype=bool)